<a href="https://colab.research.google.com/github/mwaoh/Reward-Community-Engagement/blob/main/Reward_Community_Engagement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Analysis project,Analyzing WhatsApp group chat data to draw insights.

In [44]:
#Importing libraries
import re
import regex
import pandas as pd
import numpy as np
import plotly.express as px
from collections import Counter
import matplotlib.pyplot as plt
from os import path
from PIL import Image
import datetime
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
%matplotlib inline

#Data Collection
Opening Whatsapp group chat. Go to Settings. Click on Export Chat. Click on without media.
Writing functions to extra data from the whatsapp text file


In [53]:
def startsWithDate(s):
    pattern = '^([0-2][0-9]|(3)[0-1])(\/)(((0)[0-9])|((1)[0-2]))(\/)(\d{2}|\d{4}), ([0-9][0-9]):([0-9][0-9]) -'
    result = re.match(pattern, s)
    if result:
        return True
    return False

In [54]:
def startsWithAuthor(s):
    patterns = [
        '([\w]+):',                        # First Name
        '([\w]+[\s]+[\w]+):',              # First Name + Last Name
        '([\w]+[\s]+[\w]+[\s]+[\w]+):',    # First Name + Middle Name + Last Name
        '([+]\d{2} \d{5} \d{5}):',         # Mobile Number (India)
        '([+]\d{2} \d{3} \d{3} \d{4}):',   # Mobile Number (US)
        '([+]\d{2} \d{4} \d{7})'           # Mobile Number (Europe)
    ]
    pattern = '^' + '|'.join(patterns)
    result = re.match(pattern, s)
    if result:
        return True
    return False

In [57]:
def getDataPoint(line):
    # line = 18/06/17, 22:47 - Loki: Why do you have 2 numbers, Banner?
    
    splitLine = line.split(' - ') # splitLine = ['18/06/17, 22:47', 'Loki: Why do you have 2 numbers, Banner?']
    
    dateTime = splitLine[0] # dateTime = '18/06/17, 22:47'
    
    date, time = dateTime.split(', ') # date = '18/06/17'; time = '22:47'
    
    message = ' '.join(splitLine[1:]) # message = 'Loki: Why do you have 2 numbers, Banner?'
    
    if startsWithAuthor(message): # True
        splitMessage = message.split(': ') # splitMessage = ['Loki', 'Why do you have 2 numbers, Banner?']
        author = splitMessage[0] # author = 'Loki'
        message = ' '.join(splitMessage[1:]) # message = 'Why do you have 2 numbers, Banner?'
    else:
        author = None
    return date, time, author, message

In [60]:
parsedData = [] # List to keep track of data so it can be used by a Pandas dataframe
conversationPath = '/WhatsApp Chat with  Writing & opportunities .txt' 
with open(conversationPath, encoding="utf-8") as fp:
    fp.readline() # Skipping first line of the file (usually contains information about end-to-end encryption)
        
    messageBuffer = [] # Buffer to capture intermediate output for multi-line messages
    date, time, author = None, None, None # Intermediate variables to keep track of the current message being processed
    
    while True:
        line = fp.readline() 
        if not line: # Stop reading further if end of file has been reached
            break
        line = line.strip() # Guarding against erroneous leading and trailing whitespaces
        if startsWithDate(line): # If a line starts with a Date Time pattern, then this indicates the beginning of a new message
            if len(messageBuffer) > 0: # Check if the message buffer contains characters from previous iterations
                parsedData.append([date, time, author, ' '.join(messageBuffer)]) # Save the tokens from the previous message in parsedData
            messageBuffer.clear() # Clear the message buffer so that it can be used for the next message
            date, time, author, message = getDataPoint(line) # Identify and extract tokens from the line
            messageBuffer.append(message) # Append message to buffer
        else:
            messageBuffer.append(line) # If a line doesn't start with a Date Time pattern, then it is part of a multi-line message. So, just append to buff

In [61]:
df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message'])
df.head()

,Date,Time,Author,Message
0,None,None,None,"8/25/22, 21:49 - You were added"
1,12/10/22,01:48,None,+254 704 968840: Hello anybody who can do a bi...
2,12/10/22,07:28,None,+254 770 807583: URGENTLY BUYING🔥🔥 💰💰 ✅. Write...
3,12/10/22,07:55,None,+254 706 406758: Waiting for this message
4,12/10/22,07:59,None,+1 (408) 883-8530: OFFER GET A USA 🇺🇸 NUMBER F...


In [62]:
df.describe()

,Date,Time,Author,Message
count,56,56,0,57
unique,3,51,0,28
top,12/10/22,15:39,NaN,+1 (408) 883-8530: OFFER GET A USA 🇺🇸 NUMBER F...
freq,23,2,NaN,15


In [63]:
df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'author', 'Message'])
df.head(10)

,Date,Time,author,Message
0,None,None,None,"8/25/22, 21:49 - You were added"
1,12/10/22,01:48,None,+254 704 968840: Hello anybody who can do a bi...
2,12/10/22,07:28,None,+254 770 807583: URGENTLY BUYING🔥🔥 💰💰 ✅. Write...
3,12/10/22,07:55,None,+254 706 406758: Waiting for this message
4,12/10/22,07:59,None,+1 (408) 883-8530: OFFER GET A USA 🇺🇸 NUMBER F...
5,12/10/22,08:02,None,+254 706 406758: Waiting for this message
6,12/10/22,09:33,None,+254 742 123111: Waiting for this message
7,12/10/22,09:52,None,+254 742 123111: Waiting for this message
8,12/10/22,10:44,None,+254 794 126661: Cashapp 💶 Zelle 💵 Venmo💸 Hong...
9,12/10/22,11:01,None,+254 700 917700: Essaypro Active rated 99% 4.9...


In [64]:
df.describe()

,Date,Time,author,Message
count,56,56,0,57
unique,3,51,0,28
top,12/10/22,15:39,NaN,+1 (408) 883-8530: OFFER GET A USA 🇺🇸 NUMBER F...
freq,23,2,NaN,15


In [66]:
author_value_counts = df['Author'].value_counts() # Number of messages per author
top_8_author_value_counts = author_value_counts.head(8) # Number of messages per author for the top 10 most active authors
top_8_author_value_counts.plot.barh() # Plot a bar chart using pandas built-in plotting apis

KeyError: ignored

Empty DataFrame
Columns: [Date, Time, Contact, Message]
Index: []


KeyError: ignored

In [14]:
df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Contact', 'Message']) # Initialising a pandas Dataframe.
df["Date"] = pd.to_datetime(df["Date"])

In [15]:
df.head(10)

,Date,Time,Contact,Message
0,2022-08-25,21:49,None,You were added
1,2022-12-10,01:48,+254 704 968840,Hello anybody who can do a biology lab report
2,2022-12-10,07:28,+254 770 807583,URGENTLY BUYING🔥🔥 💰💰 ✅. Writershub senior leve...
3,2022-12-10,07:55,+254 706 406758,Waiting for this message
4,2022-12-10,07:59,+1 (408) 883-8530,OFFER GET A USA 🇺🇸 NUMBER FOR ONLY KSH250.DM NOW
5,2022-12-10,08:02,+254 706 406758,Waiting for this message
6,2022-12-10,09:33,+254 742 123111,Waiting for this message
7,2022-12-10,09:52,+254 742 123111,Waiting for this message
8,2022-12-10,10:44,+254 794 126661,Cashapp 💶 Zelle 💵 Venmo💸 Hongkong bank💴 DM for...
9,2022-12-10,11:01,+254 700 917700,Essaypro Active rated 99% 4.9 320 orders with ...


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 658 entries, 0 to 657
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     658 non-null    datetime64[ns]
 1   Time     658 non-null    object        
 2   Contact  578 non-null    object        
 3   Message  658 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 20.7+ KB


In [17]:
total_messages = df.shape[0]
print(total_messages)

658


In [20]:
author_value_counts = df['Author'].value_counts() # Number of messages per author
top_10_author_value_counts = author_value_counts.head(10) # Number of messages per author for the top 10 most active authors
top_10_author_value_counts.plot.barh() # Plot a bar chart using pandas built-in plotting apis

KeyError: ignored

Loading data from WhatsApp

Data preprocessing,Cleaning the data

Data Visualization,Exploring the data 

The most user


##Establishing Occurence of words
Most Topics Discussed

Conclusion